In [86]:
# Load necessary libraries
import os
import requests
import pandas as pd
import csv

In [87]:
# Load CSV
req = requests.get(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQngt5TxTabbOavo5qHaZz5ohs9o_46sWrhQMKT5gJdedIG3Icq0qvuUX1dfdkcrmqNUxzCjOk2egSo/pub?gid=160193944&single=true&output=csv"
)
url_content = req.content
csv_file = open("../data/psychologists.csv", "wb")
csv_file.write(url_content)
csv_file.close()
CSV_PATH = "../data/psychologists.csv"

In [88]:
# Build our dataframe
df = pd.read_csv(CSV_PATH)

print('df', df)

df          Marca temporal      Nombre y Apellido  \
0    28/4/2022 14:10:20         Melisa Gaggino   
1    28/4/2022 14:12:51           Sofia Alonso   
2    28/4/2022 14:41:21                    NaN   
3    10/5/2022 10:58:00    Sofía María Bidondo   
4    10/5/2022 11:04:43  Cecilia Brittes Mazza   
..                  ...                    ...   
188  13/5/2022 21:48:20                    NaN   
189  13/5/2022 21:52:43                    NaN   
190  14/5/2022 10:46:24                    NaN   
191  14/5/2022 10:53:54                    NaN   
192  14/5/2022 12:18:20                    NaN   

           Dirección de correo electrónico Identidad de Género  \
0                 melisa.gaggino@gmail.com               Mujer   
1                    sofialonsod@gmail.com               Mujer   
2          alvarenga_german@hotmail.com.ar          No binarie   
3               licsofiambidondo@gmail.com               Mujer   
4                  cecibritmazza@gmail.com               Mujer   


In [89]:
print(df.columns)

Index(['Marca temporal', 'Nombre y Apellido',
       'Dirección de correo electrónico', 'Identidad de Género',
       'Tipo de Matrícula',
       'Número de Matrícula (en caso de tener MN y MP, poner ambas en ese orden)',
       'Añada el nombre de la institución o servicio en salud mental en caso de formar parte ',
       '¿Integra un equipo de salud con Médicx Psiquiatra?', 'Jurisdicción',
       'Acá podes especificar la localidad dentro de la provincia donde ejerces',
       'Mayor grado académico alcanzado',
       '¿En cuáles de estos modelos terapéuticos acredita formación?',
       'Formación en Perspectiva de Género/LGBTIQ+ ',
       'Otras formaciones realizadas que puedan ayudar a especificar tu perfil profesional',
       'Población de trabajo', 'Modalidad de Trabajo', 'Atención on-line',
       '¿Trabaja con Obras Sociales / Prepagas?', 'Especificar cuáles',
       '¿Facturás para realizar el reintegro en las Obras Sociales / Prepagas?',
       '¿Utiliza Lengua de Señas? (

In [90]:
# Variables
## Education

column_names = {"Mayor grado académico alcanzado": "education"}
df.rename(columns = column_names, inplace=True)

df_education = df["education"].value_counts().to_frame().reset_index()
df_education.rename(columns = {"index": "education", "education": "value"}, inplace=True)

print(df_education)

      education  value
0  Licenciatura    105
1  Especialidad     75
2      Maestría     11
3     Doctorado      2


As we can see, the most of our psychologists have a licensure degree. 

In [91]:
## Specializations
column_names = {"Otras formaciones realizadas que puedan ayudar a especificar tu perfil profesional": "specialization"}
df.rename(columns = column_names, inplace=True)

df_specialization = (
        df[["specialization"]]
        .assign(specialization=df["specialization"].str.split(","))
        .explode("specialization")
        .reset_index(drop=True)
    )

df_specialization = df_specialization.value_counts().to_frame().reset_index()
df_specialization.rename(columns = {0: "value"}, inplace=True)

print(df_specialization)

                                       specialization  value
0                     Trastornos del estado del ánimo     75
1                                                 TOC     58
2                                Estrés postraumático     50
3                                               Duelo     50
4                                 Violencia de género     44
..                                                ...    ...
80                                  Trast de Ansiedad      1
81                            Trastornos de Ansiedad.      1
82                           Trastornos de ansiedad.       1
83   Trastornos de síntomas neurológicos funcionales.      1
84                             trastorno de ansiedad       1

[85 rows x 2 columns]


In [112]:
## Therapeutic Models
column_names = {"¿En cuáles de estos modelos terapéuticos acredita formación?": "therapeutic_model"}
df.rename(columns = column_names, inplace=True)

df_therapeutic_model = (
            df[["therapeutic_model"]]
            .assign(therapeutic_model=df["therapeutic_model"].str.split(","))
            .explode("therapeutic_model")
            .reset_index(drop=True)
        )
df_therapeutic_model = df_therapeutic_model.value_counts().to_frame().reset_index()
df_therapeutic_model.rename(columns = {0: "value"}, inplace=True)


df_therapeutic_model["therapeutic_model"] = df_therapeutic_model["therapeutic_model"].apply(
    lambda row: row.strip() if row is not None else row
)

d = {'value': 'sum', 'therapeutic_model': 'first'}
df_therapeutic_model = df_therapeutic_model.groupby('therapeutic_model', as_index=False).aggregate(d).reindex(columns=df_therapeutic_model.columns).sort_values('value', ascending=False)

print(df_therapeutic_model)

                  therapeutic_model  value
5   Terapias Cognitivo Conductuales    168
0                               ACT     67
2                               DBT     47
4                       Mindfulness     37
6               Terapias Sistémicas     35
1  Análisis de la Conducta Aplicado     32
3                               FAP     13


In [113]:
## Work population
column_names = {"Población de trabajo": "work_population"}
df.rename(columns = column_names, inplace=True)

df_work_population = (
            df[["work_population"]]
            .assign(work_population=df["work_population"].str.split(","))
            .explode("work_population")
            .reset_index(drop=True)
        )

df_work_population = df_work_population.value_counts().to_frame().reset_index()
df_work_population.rename(columns = {0: "value"}, inplace=True)

df_work_population["work_population"] = df_work_population["work_population"].apply(
    lambda row: row.strip() if row is not None else row
)

d = {'value': 'sum', 'work_population': 'first'}
df_work_population = df_work_population.groupby('work_population', as_index=False).aggregate(d).reindex(columns=df_work_population.columns).sort_values('value', ascending=False)

print(df_work_population)

             work_population  value
0                    Adultxs    183
1            Adultxs Mayores     43
2             Infantojuvenil     85
3  Personas con discapacidad     21
